# Own your health app!

Trying to show a simple PoC where I give it a medical test document and then ask 
some structured responses from it. 

For our real app, we will have 10's or 100's of such documents per individual 
and each document could be 10's to 100's of pages. This is because someone's diagnostic 
journey e.g. cancer is spread across tests and visits to 100's of institutions before they
get refered to a large cancer center oncologist and this oncology team has to make sense of all you have
endured {treatment, outcomes, discharge summaries} to give you the right next treatment when 
you arrive at their doorstep.


For now success will be if for a single report I am able to get back all the
diagnostic tests done without missing. It tends to miss a few and on repeated nudging since 
I know the answer, pull out more and more tests missed previously.
Also, as a bonus what I would really really like would be I ask a question with a schema 
and it returns *all* the elements in the same schema.


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader

# Right now this does not do directory, it takes one file at a time, could this be a limitation later !?!?

## First approach is doing this with one big gulp, no splitting and using function calls to structure

In [ ]:
## Core algorithm which parses the PDF file and structres the output.
# the plus of using pedantic models is yo can force what you want the return obect to look like
# pedantic then does the validation to make sure!
# In the future we can make this configurable!

#TODO: Adding a list object into the pedayntic object which itself is an object?


from pydantic import BaseModel, Field
from langchain.chains.openai_functions.extraction import _get_extraction_function
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

doc = UnstructuredPDFLoader(file_path="/Users/vinayak/projects/medical_records_parser/data/MinnieMouseReport.pdf")
# Above I read the whole thing as ONE large blob, this was possible since the file is only 7 pages!
# if the file becomes too large, this is not possible.
docs = doc.load()


query = """
You are an expert medical transcriber. Please give me back a table of all the analytes measured. The table should have following columns: analyte_measured, result, reference_interval, unit, notes. 
If a patricular column does not exist please say NA.
Please double check your work and do not miss any analyte.
"""


class Analyte(BaseModel):
    """Information about an analyte."""
    analyte_measured: str
    result: str
    reference_interval: str
    unit: str
    notes: str

openai_function = _get_extraction_function(Analyte.schema())

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo-16k")

prompt = ChatPromptTemplate.from_messages([
    ("system", query),
    ("user", "{doc}")
])


output_parser = JsonKeyOutputFunctionsParser(key_name="info")
chain = prompt | model.bind(functions=[openai_function], function_call={"name": "information_extraction"}) | output_parser

response = chain.invoke({"doc": docs[0].page_content})

In [ ]:
pd.DataFrame(response) # Amit/Guy can you please see if the output is correct?

## Second approach is doing the same thing as above but one page at a time since sometimes the file might be too large to fit context size of the model.

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
doc = UnstructuredPDFLoader(file_path="/Users/vinayak/projects/medical_records_parser/data/MinnieMouseReport.pdf", mode="paged")
# Above is the key difference where it is loading the data as "paged" mode.

docs = doc.load()

query = """
Please give me back a table of all the analytes measured. The table should have following columns: analyte_measured, result, reference_interval, unit, notes. 
If a patricular column does not exist please say NA.
Please double check your work and do not miss any analyte.
"""

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

model = ChatOpenAI(model="gpt-3.5-turbo-16k")

prompt = ChatPromptTemplate.from_messages([
    ("system", query),
    ("user", "{doc}")
])

# This is because of what _get_extraction_function does
output_parser = JsonKeyOutputFunctionsParser(key_name="info")

chain = prompt | model.bind(functions=[openai_function], function_call={"name": "information_extraction"}) | output_parser


responses = chain.batch([{"doc": d.page_content} for d in docs], {"max_concurrency": 5})

extracted_by_function_call = []
for response in responses:
    extracted_by_function_call.extend(response)
    


In [ ]:
pd.DataFrame(extracted_by_function_call)

## Now for the third approach, this is a VERY large document of 130+ pages including mishmash of diffrent kind of reports since it has come from an EMR dump (likely EPIC)

In [ ]:
doc = UnstructuredPDFLoader(file_path="/Users/vinayak/projects/kaiser/data/Barbara/UCLA Health.pdf")
docs = doc.load()

In [ ]:
#Showing the entire contents of the document
docs[0].page_content

In [ ]:
# we need some kind of splitter which closely resembles where records start and end
# I am using the default on, which is sub optimal and does a lot of repeats and not so useful summarization too!

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 10000,
    chunk_overlap  = 500,
    length_function = len,
    is_separator_regex = False,
)

doc = UnstructuredPDFLoader("/Users/vinayak/projects/kaiser/data/Barbara/UCLA Health.pdf")

docs = doc.load_and_split(text_splitter)

print("Number of splits %d"%(len(docs)))


### Notice I also changed the question from parsing a diagnostic report to parsing medical visits. Likely we will have to do a hybrid where we first split the very large document into different pieces and for each piece parse what is relevant (diagnostic report vs visits vs pathology report)

In [ ]:
query = """
You are an expert medical transcriber and can transcribe electronic health records with great skill.
Please give me back a table of all the visits from the patient. Columns to return are:
patient_name, date_of_visit, category, provider, institution, brief_summary
The category can only be one of the following values: LAB_REPORT, PATHOLOGY, RADIOLOGY, PROCEDURE, DIAGNOSTIC_TEST, ROUTINE_VISIT
If a patricular column does not exist please say NA.
Please double check your work and do not miss any visits.
"""


from pydantic import BaseModel, Field
from langchain.chains.openai_functions.extraction import _get_extraction_function

class Visit(BaseModel):
    """Information about visit to the medical facility."""
    patient_name: str
    date_of_visit: str
    category: str
    provider: str
    institution: str
    brief_summary: str

openai_function = _get_extraction_function(Visit.schema())

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo-16k")

prompt = ChatPromptTemplate.from_messages([
    ("system", query),
    ("user", "{doc}")
])

chain = prompt | model.bind(functions=[openai_function], function_call={"name": "information_extraction"}) | output_parser

# Make subset of docs below (8) so I don't become bankrupt! with openAI bills

responses = chain.batch([{"doc": d.page_content} for d in docs], {"max_concurrency": 5})


## The above code aks a question per subset of the data (according to the split which is 10k). This means it will have
## answers per split. The response object is list of response, each response hiving a list of dicts

In [ ]:
# we need to flatten the list

flattened_list = list()
for d in responses:
    flattened_list.extend(d)

flattened_list

In [ ]:
import pandas as pd
df = pd.DataFrame.from_records(flattened_list)
df


In [ ]:
#Doing cleanup to remove junk, this is because our parser is not yet good enough.

df1 = df[(df.patient_name != 'NA')]
df1 = df1[(df1.category != 'NA')]

df1['date_cleanedup']= pd.to_datetime(df1['date_of_visit'], format='mixed')
df1['final_date'] = df1['date_cleanedup'].apply(lambda x: x.strftime('%B %d, %Y'))

df1.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)

df1.sort_values(by='date_cleanedup').to_csv('/Users/vinayak/projects/df_to_test.tsv', sep="\t", index=False)




In [ ]:
df1.rename(columns={'final_date': 'title','institution': 'cardTitle', 'category': 'cardSubtitle', 'brief_summary': 'cardDetailedText' }, inplace=True)
cols_needed = ['title', 'cardTitle', 'cardSubtitle', 'cardDetailedText']
df1[cols_needed].to_dict('records')

# Scratch space

### The stuff below is working but chunking is per page

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
doc = UnstructuredPDFLoader(file_path="/Users/vinayak/projects/kaiser/data/Barbara/UCLA Health.pdf", mode="paged")

docs = doc.load()

query = """
Please give me back a table of all the visits from the patient. Columns to return are:
visit_date, visit_reason, visit_department, visit_summary
If a patricular column does not exist please say NA.
Please double check your work and do not miss any visits.
"""

class Visit(BaseModel):
    """Information about visit to the medical facility."""
    visit_date: str
    visit_reason: str
    visit_department: str
    visit_summary: str



openai_function = _get_extraction_function(Visit.schema())

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

model = ChatOpenAI(model="gpt-3.5-turbo-16k")

prompt = ChatPromptTemplate.from_messages([
    ("system", query),
    ("user", "{doc}")
])

# This is because of what _get_extraction_function does
output_parser = JsonKeyOutputFunctionsParser(key_name="info")

chain = prompt | model.bind(functions=[openai_function], function_call={"name": "information_extraction"}) | output_parser


responses = chain.batch([{"doc": d.page_content} for d in docs], {"max_concurrency": 5})